# About: HTTPS接続に変更する
---
Moodleへの接続を HTTP から HTTPS に変更します。

## 概要

「021-Moodleコンテナの起動.ipynb」で構築した環境ではMoodleにHTTP(80)で接続する設定となっています。これをHTTPS(443)で接続するように変更します。

![処理の流れ](images/moodle-071-01.png)

設定手順の主な流れを以下に示します。

1. HTTPサーバで利用する証明書と秘密鍵を配置する
1. httpdの設定ファイルを配置する
1. httpdコンテナの構成を記述した `docker-compose.yml` を配置する
1. 新たな構成でコンテナを起動しなおす

### コンテナ構成について

「021-Moodleコンテナの起動.ipynb」で構築した環境は次図のようなコンテナ構成となっています。

![変更前の構成](images/moodle-071-02.png)

 MoodleコンテナではMoodle をHTTP(80)で外部に公開する Apache HTTP server(httpd)と、MoodleのPHPファイルを実行するphp-fpmが実行されています。httpdとphp-fpmはUNIXドメインソケットを経由して通信を行っています。

HTTPS(443)でMoodleを公開するために、先ほど示したコンテナ構成を次図のように変更します。

![変更後の構成](images/moodle-071-03.png)

MoodleをHTTPS(443)で公開するためのhttpdコンテナをMoodleコンテナの前段に配置しています。httpdコンテナにはサーバ証明書をbindマウントし、かつmod_sslを有効にしてApache HTTP server(httpd)がHTTPS(443)で公開できるように設定します。

httpdコンテナで受け付けた外部からのリクエストはUNIXドメインソケットを経由してMoodleコンテナのphp-fpmで処理します。この連携を行うためにホスト環境にあるUNIXドメインソケットのファイルをhttpdコンテナとMoodleコンテナからbindマウントして、それぞれのコンテナから同一のソケットが見えるように設定します。 この構成では元々Moodleコンテナで実行していたhttpdは不要なので停止させます。

## 準備

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

### 前提条件の確認

このNotebookを実行するための前提条件を満たしていることを確認します。

前提となる条件を以下に示します。

* 構築した際のパラメータが group_vars ファイルに記録されていること
* Ansibleから操作可能であること
* Ansibleから管理者権限でコマンドを実行できること
* Moodleコンテナが実行中であること
* Moodleコンテナでphp-fpmが実行されていること

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

対象となるホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

管理者権限でコマンド実行可能であることを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

Moodleコンテナが実行中であることを確認します。

In [ ]:
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    docker compose ps --status running moodle | grep moodle'

Moodleコンテナでphp-fpmが実行されていることを確認します。

In [ ]:
!ansible {target_group} -m shell -a 'chdir=/srv/moodle \
    docker compose exec moodle supervisorctl status || \
    docker compose exec moodle ls /run/php-fpm/www.sock'

## パラメータ設定

### MoodleのURL

MoodleではサイトのURLを`config.php`の`$CFG->wwwroot`に設定しています。HTTPS接続にすることによりURLが変更される場合はその値を指定します。

> 公開するURLが現在の設定値から変更がない場合はこの節の操作をスキップして「4. メンテナンスモードへの切り替え」に進んでください。

現在設定されている URL の値を確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
print(gvars["moodle_url"])

新たなURLの値を次のセルで指定してください。

In [ ]:
# (例)
# moodle_url = 'https://moodle.example.org'

moodle_url = 

#### パラメータの保存

この節で指定したパラメータの値をファイルに保存します。

値の保存を行う前に入力されたパラメータに対して簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/utils.py
check_parameters(
    'moodle_url',
    nb_vars=locals(),
)

パラメータの値を group_vars ファイルに保存します。

In [ ]:
from urllib.parse import urlparse
%run scripts/group.py

res = urlparse(moodle_url)
update_group_vars(
    target_group,
    _file="30-httpd.yml",
    moodle_url=moodle_url,
    server_name=res.netloc,
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-httpd.yml

## メンテナンスモードへの切り替え

Moodleコンテナが一時停止することになるので、メンテナンスモードに切り替えます。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

## サーバ証明書の配置

httpdコンテナで使用するサーバ証明書を配置します。

Apache HTTP serverで利用する証明書と秘密鍵を配置します。

> ここで指定するサーバ証明書と秘密鍵は事前にこのNotebook環境にアップロードしておいてください。

ここで指定した証明書、秘密鍵は構築環境に配置されhttpdコンテナにbindマウントされます。配置先のパスとコンテナにおけるパスの関係を次表に示します。

|構築環境の配置先となるパス|コンテナでのパス|説明|Apache HTTP serverでの設定項目|
|:---|:---|:---|:---|
|/srv/moodle/httpd/conf/httpd/certs/server.crt|/etc/pki/tls/certs/server.crt|Apache HTTP serverのサーバ証明書|SSLCertificateFile|
|/srv/moodle/httpd/conf/httpd/certs/server.key|/etc/pki/tls/private/server.key|Apache HTTP serverの秘密鍵|SSLCertificateKeyFile|


### 配置するファイルの指定

構築環境に配置するファイルを指定します。

 ####  サーバ証明書の指定
 
 サーバ証明書のパスを次のセルで指定してください。ここで指定するパスはNotebook環境におけるサーバ証明書のパスになります。

In [ ]:
# (例)
# moodle_cert_path = './certs/server.crt'

moodle_cert_path = 

 証明書のフォーマットはPEMであることを想定しています。 またサーバ証明書に中間CA証明書を連結する場合は[電子証明書発行サービス-サーバ動作事例集](https://certs.nii.ac.jp/manual/manuals/reports/03)で説明されているように、「サーバ証明書」、「中間CA証明書」の順に記載してください。

#### 秘密鍵の指定

秘密鍵のパスを次のセルで指定してください。ここで指定するパスはNotebook環境における秘密鍵のパスになります。

In [ ]:
# (例)
# moodle_private_key_path = './certs/server.key'

moodle_private_key_path = 

#### パラメータの保存

指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py
from pathlib import Path

update_group_vars(
    target_group,
    _file="30-httpd.yml",
    moodle_cert_path=str(Path(moodle_cert_path).resolve()),
    moodle_private_key_path=str(Path(moodle_private_key_path).resolve()),
    httpd="httpd",
)

`group_vars`ファイルの内容を確認してみます。

In [ ]:
!cat group_vars/{target_group}/30-httpd.yml

### 証明書と秘密鍵を構築環境に配置する

次のセルを実行すると証明書、秘密鍵を構築環境に配置する ansible の playbook を実行します。

実際の操作を行う前にチェックモードで ansible を実行します。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-httpd-certs.yml || true

実際に証明書の配置を行います。

In [ ]:
!ansible-playbook -l {target_group} -v playbooks/deploy-httpd-certs.yml

サーバ証明書がMoodle環境に配置されたことを確認するために、証明書の内容を表示してみます。

In [ ]:
!ansible {target_group} -a \
    'openssl x509 -in /srv/moodle/httpd/conf/httpd/certs/server.crt \
    -noout -text'

配置した秘密鍵を表示します。

In [ ]:
!ansible {target_group} -b -a \
    'openssl rsa -in /srv/moodle/httpd/conf/httpd/certs/server.key \
    -noout -text'

## 設定ファイルの配置

httpdコンテナで実行するApache HTTP serverの設定ファイルを構築環境のホスト側に配置します。

bind mountによって配置する設定ファイルのホスト環境におけるパスとコンテナ環境におけるパスの対応関係を以下に示します。

|コンテナ名|コンテナ環境のパス|ホスト環境のパス|
|:---|:---|:---|
|httpd|/etc/httpd/conf/httpd.conf|/srv/moodle/httpd/conf/httpd/conf/httpd.conf|
|httpd|/etc/httpd/conf.d/|/srv/moodle/moodle/conf/httpd/conf.d/|
|httpd|/etc/httpd/conf.modules.d/|/srv/moodle/httpd/conf/httpd/conf.modules.d/|

実際の操作を行う前にチェックモードで ansible を実行します。

In [ ]:
!ansible-playbook -l {target_group} -CDv playbooks/deploy-httpd-conf.yml

実際に設定ファイルの配置を行います。

In [ ]:
!ansible-playbook -l {target_group} -v playbooks/deploy-httpd-conf.yml

## コンテナ構成の変更

MoodleをHTTPSで公開するためのApache HTTP Serverのコンテナを追加する構成変更を行います。

### docker-compose.yml の配置

このNotebookの設定手順ではHTTPS接続に対応するためのコンテナを追加する変更を行います。新たなコンテナ構成に対応する`docker-compose.yml`を構築環境に配置します。ここで配置する`docker-compose.yml`は元の設定に対して以下に示す変更をおこなっています。

* Apache HTTP Serverのmod_ssl を有効にしたhttpdコンテナを追加する
* moodleコンテナで実行している `php-fpm` のソケットを httpd コンテナを共有するように bind マウントを設定する
* moodleコンテナの httpd サービスを無効化する
* moodleコンテナの`config.php`で設定している `$CFG->wwwroot` の値を新たな値に変更する

実際に`docker-compose.yml`を配置する前にチェックモードで ansible を実行します。

In [ ]:
!ansible {target_group} -CDv -m template \
    -a 'src=template/docker-compose.yml.j2 \
    dest=/srv/moodle/docker-compose.yml backup=yes'

`docker-compose.yml`を構築環境に配置します。

In [ ]:
!ansible {target_group} -m template \
    -a 'src=template/docker-compose.yml.j2 \
    dest=/srv/moodle/docker-compose.yml backup=yes'

### コンテナ設定の変更を反映する

コンテナ設定の変更反映をするためにコンテナを再起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d --remove-orphans'

変更後のコンテナの状態を確認してみます。`State` の値が `Up` となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

`State`が`Up`となっていない場合はコンテナのログなどを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose logs --tail=50 httpd'

## メンテナンスモードの解除

設定変更が完了したのでメンテナンスモードを解除します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

Moodle環境にアクセスして利用可能であることを確認します。

次のセルを実行するとMoodleのアドレスを表示します。表示されたリンクをクリックしてMoodleにアクセスできることを確認してください。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
print(gvars["moodle_url"])